In [1]:
import json
import time
import requests
from bs4 import BeautifulSoup

# 1. Ruta de entrada/salida: ajusta según corresponda
INPUT_PATH  = "/home/lenovo/Documentos/repos/final-project-cinedive/static/data/graph_for_project.json"
OUTPUT_PATH = "/home/lenovo/Documentos/repos/final-project-cinedive/static/data/graph_for_project_con_country.json"

# 2. Cabeceras HTTP para simular un navegador
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}

def obtener_country_desde_imdb(imdb_id):
    """
    Dado un IMDb ID (por ejemplo 'tt0015864'), retorna el país de origen
    tal como lo muestra IMDb en la sección 'Country of origin'.
    Si no se encuentra, retorna None.
    """
    url = f"https://www.imdb.com/title/{imdb_id}/"
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"  [ERROR] no se pudo obtener {imdb_id}: {e}")
        return None

    soup = BeautifulSoup(resp.text, "html.parser")

    # En la página de IMDb, el bloque de "Country of origin" suele tener:
    # <li><span class="ipc-metadata-list-item__label">Country of origin</span>
    #     <div class="ipc-metadata-list-item__content-container">United States</div></li>
    #
    # Buscamos la etiqueta que contenga exactamente "Country of origin" y extraemos el siguiente contenido.
    etiqueta = soup.find(
        lambda tag:
            tag.name == "span"
            and "Country of origin" in tag.get_text()
            and "ipc-metadata-list-item__label" in tag.get("class", [])
    )
    if not etiqueta:
        return None

    # El contenedor de contenido suele ser un hermano o elemento padre
    padre = etiqueta.find_parent("li")
    if not padre:
        return None

    contenido = padre.find("div", class_="ipc-metadata-list-item__content-container")
    if not contenido:
        return None

    texto = contenido.get_text(strip=True)
    return texto or None

def actualizar_countries_en_json(input_path, output_path):
    # 1. Leer JSON original
    with open(input_path, "r", encoding="utf-8") as f:
        grafo = json.load(f)

    nodos = grafo.get("nodes", [])
    total = len(nodos)
    enviados = 0

    for nodo in nodos:
        # Sólo nos interesan nodos de tipo 'movie'
        if nodo.get("type") != "movie":
            continue

        country_actual = nodo.get("country_origin")
        # Detectar valores “vacíos”: None, cadena vacía, "NaN", etc.
        if country_actual is None or (isinstance(country_actual, str) and country_actual.strip().lower() in ("", "nan", "null")):
            imdb_id = nodo.get("id")
            if not imdb_id:
                continue

            # 2. Hacer scraping en IMDb
            pais = obtener_country_desde_imdb(imdb_id)
            if pais:
                nodo["country_origin"] = pais
                print(f"[OK] {imdb_id} → {pais}")
            else:
                print(f"[WARN] {imdb_id} → no se halló country")

            enviados += 1
            # Muy importante: esperar unos segundos para no sobrecargar IMDb
            time.sleep(1.5)

    # 3. Guardar nuevo JSON con todos los movies actualizados
    with open(output_path, "w", encoding="utf-8") as f_out:
        json.dump(grafo, f_out, ensure_ascii=False, indent=2)

    print(f"\nProceso finalizado. Se procesaron {enviados} nodos de {total}.")

if __name__ == "__main__":
    actualizar_countries_en_json(INPUT_PATH, OUTPUT_PATH)


[OK] tt0015864 → United States
[OK] tt0017743 → United States
[OK] tt0017806 → United States
[OK] tt0017822 → United States
[OK] tt0017979 → United States
[OK] tt0018037 → United States
[OK] tt0018129 → United States
[OK] tt0018253 → United States
[OK] tt0018379 → United States
[OK] tt0018389 → United States
[OK] tt0018429 → United States


KeyboardInterrupt: 